In [1]:
%load_ext autoreload
%autoreload 2

import torch
from mvn.utils import multiview

from mvn.datasets.cmu import CMUSceneDataset
from mvn.datasets.human36m import Human36MMultiViewDataset, Human36MSingleViewDataset
from torch.utils.data import DataLoader

import os
from itertools import islice
from tqdm import tqdm_notebook
import pickle


import numpy as np
import cv2
import time
import sys

from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from mvn.utils.img import image_batch_to_numpy, denormalize_image
from mvn.models.temporal import TemporalModel, TemporalModelBase
from mvn.models.basic import VolumetricTriangulationNet, VolumetricTemporalTriangulationNet
from mvn.utils.multiview import project_3d_points_to_image_plane_without_distortion
from mvn.utils.vis import draw_2d_pose

from mvn.utils import img
from mvn.utils import multiview
from mvn.utils import volumetric
from mvn.utils import op
from mvn.utils import vis
from mvn.utils import misc
from mvn.utils import cfg

from time import time

from easydict import EasyDict

from mvn.datasets import cmu, kth
from mvn.datasets import utils as dataset_utils
from mvn.datasets.human36m import Human36MMultiViewDataset, Human36MSingleViewDataset

from train import setup_human36m_dataloaders

from IPython.core.debugger import set_trace

from matplotlib import pyplot as plt
from warnings import filterwarnings
filterwarnings("ignore")

%matplotlib inline

retval = {
    'subject_names': ['S1', 'S5', 'S6', 'S7', 'S8', 'S9', 'S11'],
    'camera_names': ['54138969', '55011271', '58860488', '60457274'],
    'action_names': [
        'Directions-1', 'Directions-2',
        'Discussion-1', 'Discussion-2',
        'Eating-1', 'Eating-2',
        'Greeting-1', 'Greeting-2',
        'Phoning-1', 'Phoning-2',
        'Posing-1', 'Posing-2',
        'Purchases-1', 'Purchases-2',
        'Sitting-1', 'Sitting-2',
        'SittingDown-1', 'SittingDown-2',
        'Smoking-1', 'Smoking-2',
        'TakingPhoto-1', 'TakingPhoto-2',
        'Waiting-1', 'Waiting-2',
        'Walking-1', 'Walking-2',
        'WalkingDog-1', 'WalkingDog-2',
        'WalkingTogether-1', 'WalkingTogether-2']
}
# import nonechucks

device = torch.cuda.current_device()

In [2]:
# experiment_root = '../../logs/vanilla_v2v/h36_sv_dist_default_bs2-5@10.10.2019-17:58:03'
# config_path = experiment_root + '/config.yaml'

config_path = './experiments/paper/human36m/vol_2stage2/human36m_v2v_softmax.yaml'

config = cfg.load_config(config_path)

# config.opt.val_batch_size=1
# config.dataset.val.retain_every_n_frames_in_test = 1

train_loader, val_loader = setup_human36m_dataloaders(config)

In [3]:
model = VolumetricTemporalTriangulationNet(config.model).to(device)

Loading pretrained weights from: /media/hpc2_storage/ibulygin/multi-view-net-files/pose_net/pose_resnet_4.5_pixels_human36m.pth
Reiniting first 16/17 filters: module.final_layer.weight
Reiniting first 16 values in: module.final_layer.bias
Successfully loaded pretrained weights


In [4]:
for batch in train_loader:
    break

In [5]:
images_batch, keypoints_3d_batch_gt, keypoints_3d_validity_batch_gt, proj_matricies_batch = dataset_utils.prepare_batch(batch, device, config)


In [6]:
keypoints = batch['keypoints_3d']
keypoints = torch.from_numpy(keypoints).to(device)
volume_size = 64
cuboid_side = 2500
cuboids = []
sides = np.array([cuboid_side, cuboid_side, cuboid_side])

# keypoints = keypoints[:,1]

# OLD

In [7]:
coord_volumes_batch = model.get_coord_volumes(cuboid_side, volume_size, device, keypoints)[0]
coord_volumes_batch = coord_volumes_batch.view(-1, 64,64,64,3)

In [8]:
images_batch = torch.tensor(images_batch, dtype=torch.float, device=device)
image_shape = images_batch.shape[-3:]
bs,dt = images_batch.shape[:2]

In [9]:
heatmaps, features, tri_confidences, vol_confidences = model.backbone(images_batch.view(-1,*image_shape))

In [10]:
features = model.process_features(features)
features = features.unsqueeze(1)

In [11]:
features.shape

torch.Size([6, 1, 32, 96, 96])

In [10]:
proj_matricies_batch = proj_matricies_batch.view(-1, 1, *proj_matricies_batch.shape[-2:])

In [11]:
proj_matricies_batch.shape

torch.Size([6, 1, 3, 4])

In [13]:
coord_volumes_batch.shape

torch.Size([6, 64, 64, 64, 3])

In [14]:
coord_volumes_batch.shape[:-2]

torch.Size([6, 64, 64])

In [18]:
def lift_heatmap_to_volume_2_old(heatmaps_batch, proj_matricies_batch, coord_volumes_batch, volume_aggregation_method='sum', vol_confidences=None, timestep_multiplier = 1.0):
    device = heatmaps_batch.device
    batch_size, n_views, n_joints, heatmap_shape = heatmaps_batch.shape[0], heatmaps_batch.shape[1], heatmaps_batch.shape[2], tuple(heatmaps_batch.shape[3:])
    volume_shape = coord_volumes_batch.shape[1:4]
    
    # shape (bs, 32, 64, 64, 64)
    volume_batch = torch.zeros(batch_size, n_joints, *volume_shape, device=device) if (volume_aggregation_method != 'no_aggregation') else []

    # TODO: speed up this this ineffective loop
    for batch_i in range(batch_size):
        coord_volume = coord_volumes_batch[batch_i]
        grid_coord = coord_volume.reshape((-1, 3))

        volume_batch_to_aggregate = torch.zeros(n_views, n_joints, *volume_shape, device=device)

        for view_i in range(n_views):
            heatmap = heatmaps_batch[batch_i, view_i]
            heatmap = heatmap.unsqueeze(0)

            grid_coord_proj = multiview.project_3d_points_to_image_plane_without_distortion(
                proj_matricies_batch[batch_i, view_i], grid_coord, convert_back_to_euclidean=False
            )

            invalid_mask = grid_coord_proj[:, 2] <= 0.0  # depth must be larger than 0.0

            grid_coord_proj[grid_coord_proj[:, 2] == 0.0, 2] = 1.0  # not to divide by zero
            grid_coord_proj = multiview.homogeneous_to_euclidean(grid_coord_proj)

            # transform to [-1.0, 1.0] range
            grid_coord_proj_transformed = torch.zeros_like(grid_coord_proj)
            grid_coord_proj_transformed[:, 0] = 2 * (grid_coord_proj[:, 0] / heatmap_shape[0] - 0.5)
            grid_coord_proj_transformed[:, 1] = 2 * (grid_coord_proj[:, 1] / heatmap_shape[1] - 0.5)
            grid_coord_proj = grid_coord_proj_transformed

            # NOTE: it's hack solution for now. In grid_proj appears inf values because of dividing by zero when converting from homogenious to euclidean
            #grid_coord_proj = torch.clamp(grid_coord_proj, -1.01, 1.01)

            # prepare to F.grid_sample
            grid_coord_proj = grid_coord_proj.unsqueeze(1).unsqueeze(0)
            current_volume = F.grid_sample(heatmap, grid_coord_proj)

            # zero out non-valid points
            current_volume = current_volume.view(n_joints, -1)
            current_volume[:, invalid_mask] = 0.0

            # reshape back to volume
            current_volume = current_volume.view(n_joints, *volume_shape)

            # collect
            volume_batch_to_aggregate[view_i] = current_volume

        # agregate resulting volume
        if volume_aggregation_method.startswith('conf'):
            volume_batch[batch_i] = (volume_batch_to_aggregate * vol_confidences[batch_i].view(n_views, n_joints, 1, 1, 1)).sum(0)
        elif volume_aggregation_method == 'sum':
            volume_batch[batch_i] = volume_batch_to_aggregate.sum(0)
        elif volume_aggregation_method == 'max':
            volume_batch[batch_i] = volume_batch_to_aggregate.max(0)[0]
        elif volume_aggregation_method == 'softmax':
            volume_batch_to_aggregate_softmin = volume_batch_to_aggregate.clone()
            volume_batch_to_aggregate_softmin = volume_batch_to_aggregate_softmin.view(n_views, -1)
            volume_batch_to_aggregate_softmin = nn.functional.softmax(volume_batch_to_aggregate_softmin, dim=0)
            volume_batch_to_aggregate_softmin = volume_batch_to_aggregate_softmin.view(n_views, n_joints, *volume_shape)

            volume_batch[batch_i] = (volume_batch_to_aggregate * volume_batch_to_aggregate_softmin).sum(0)

        elif volume_aggregation_method == 'no_aggregation':
            volume_batch.append(volume_batch_to_aggregate) 

        else:
            raise ValueError("Unknown volume_aggregation_method: {}".format(volume_aggregation_method))
               
    return volume_batch

In [16]:
proj_matricies_batch.shape, features.shape, coord_volumes_batch.shape 

(torch.Size([6, 1, 3, 4]),
 torch.Size([6, 1, 32, 96, 96]),
 torch.Size([6, 64, 64, 64, 3]))

In [19]:
volumes  = lift_heatmap_to_volume_2_old(features,  
                            proj_matricies_batch, 
                            coord_volumes_batch,
                            volume_aggregation_method='no_aggregation',
                            vol_confidences=vol_confidences,
                            timestep_multiplier = model.timestep_multiplier
                            )

In [32]:
coord_volumes_batch.shape

torch.Size([6, 64, 64, 64, 3])

In [33]:
features.shape

torch.Size([6, 1, 32, 96, 96])

In [20]:
volumes[0].shape

torch.Size([1, 32, 64, 64, 64])

In [ ]:
torch.cat()

In [34]:
model.timestep_multiplier

1.0

# NEW

In [ ]:
def lift_heatmap_to_volume_2(heatmaps_batch, proj_matricies_batch, coord_volumes_batch, volume_aggregation_method='sum', vol_confidences=None, timestep_multiplier = 1.0):
    device = heatmaps_batch.device
    batch_size, n_views, n_joints, heatmap_shape = heatmaps_batch.shape[0], heatmaps_batch.shape[1], heatmaps_batch.shape[2], tuple(heatmaps_batch.shape[3:])
    volume_shape = coord_volumes_batch.shape[1:4]
    
    # shape (bs, 32, 64, 64, 64)
    volume_batch = torch.zeros(batch_size, n_joints, *volume_shape, device=device) if (volume_aggregation_method != 'no_aggregation') else []

    # TODO: speed up this this ineffective loop
    for batch_i in range(batch_size):
        coord_volume = coord_volumes_batch[batch_i]
        grid_coord = coord_volume.reshape((-1, 3))

        volume_batch_to_aggregate = torch.zeros(n_views, n_joints, *volume_shape, device=device)

        for view_i in range(n_views):
            heatmap = heatmaps_batch[batch_i, view_i]
            heatmap = heatmap.unsqueeze(0)

            grid_coord_proj = multiview.project_3d_points_to_image_plane_without_distortion(
                proj_matricies_batch[batch_i, view_i], grid_coord, convert_back_to_euclidean=False
            )

            invalid_mask = grid_coord_proj[:, 2] <= 0.0  # depth must be larger than 0.0

            grid_coord_proj[grid_coord_proj[:, 2] == 0.0, 2] = 1.0  # not to divide by zero
            grid_coord_proj = multiview.homogeneous_to_euclidean(grid_coord_proj)

            # transform to [-1.0, 1.0] range
            grid_coord_proj_transformed = torch.zeros_like(grid_coord_proj)
            grid_coord_proj_transformed[:, 0] = 2 * (grid_coord_proj[:, 0] / heatmap_shape[0] - 0.5)
            grid_coord_proj_transformed[:, 1] = 2 * (grid_coord_proj[:, 1] / heatmap_shape[1] - 0.5)
            grid_coord_proj = grid_coord_proj_transformed

            # NOTE: it's hack solution for now. In grid_proj appears inf values because of dividing by zero when converting from homogenious to euclidean
            #grid_coord_proj = torch.clamp(grid_coord_proj, -1.01, 1.01)

            # prepare to F.grid_sample
            grid_coord_proj = grid_coord_proj.unsqueeze(1).unsqueeze(0)
            current_volume = F.grid_sample(heatmap, grid_coord_proj)

            # zero out non-valid points
            current_volume = current_volume.view(n_joints, -1)
            current_volume[:, invalid_mask] = 0.0

            # reshape back to volume
            current_volume = current_volume.view(n_joints, *volume_shape)

            # collect
            volume_batch_to_aggregate[view_i] = current_volume

        # agregate resulting volume
        if volume_aggregation_method.startswith('conf'):
            volume_batch[batch_i] = (volume_batch_to_aggregate * vol_confidences[batch_i].view(n_views, n_joints, 1, 1, 1)).sum(0)
        elif volume_aggregation_method == 'sum':
            volume_batch[batch_i] = volume_batch_to_aggregate.sum(0)
        elif volume_aggregation_method == 'max':
            volume_batch[batch_i] = volume_batch_to_aggregate.max(0)[0]
        elif volume_aggregation_method == 'softmax':
            volume_batch_to_aggregate_softmin = volume_batch_to_aggregate.clone()
            volume_batch_to_aggregate_softmin = volume_batch_to_aggregate_softmin.view(n_views, -1)
            volume_batch_to_aggregate_softmin = nn.functional.softmax(volume_batch_to_aggregate_softmin, dim=0)
            volume_batch_to_aggregate_softmin = volume_batch_to_aggregate_softmin.view(n_views, n_joints, *volume_shape)

            volume_batch[batch_i] = (volume_batch_to_aggregate * volume_batch_to_aggregate_softmin).sum(0)

        elif volume_aggregation_method == 'no_aggregation':
            volume_batch.append(volume_batch_to_aggregate.view(-1, *volume_shape)*(timestep_multiplier if view_i != n_views//2 else 1.0)) 

        else:
            raise ValueError("Unknown volume_aggregation_method: {}".format(volume_aggregation_method))
               
    return volume_batch

In [ ]:
batch_shorted =  batch.copy()
batch_shorted['keypoints_3d'] = batch_shorted['keypoints_3d'][:,0]

In [ ]:
model = EasyDict({'training': True,
                  'rotation':False,
                  'kind':'mpii',
                  'transfer_cmu_to_human36m':False,
                  'use_precalculated_pelvis': False,
                  'use_gt_pelvis': True,
                  'use_separate_v2v_for_basepoint': False})


In [ ]:
grid_t = []
grid_old_t = []
sizes = list(range(1,40))

for i in tqdm_notebook(sizes):
    
    keypoints = np.random.randn(i,17,4)
    batch_shorted['keypoints_3d'] = keypoints
    
    
    t1 = time()
    grid = get_coord_volumes(model, 
                            cuboid_side, 
                            volume_size, 
                            device, 
                            keypoints = keypoints)

    t2 = time()
    grid_old = get_coord_volumes_old(model, 
                                    batch_shorted, 
                                    cuboid_side, 
                                    volume_size, 
                                    device, 
                                    determine_base_point = True, 
                                    tri_keypoints_3d = None
                                    )

    t3 = time()
    
    grid_t += [round(t2-t1,4)]
    grid_old_t += [round(t3-t2,4)]
    
    assert bool(torch.all(grid == grid_old[0]))

In [ ]:
xticks = sizes
plt.figure(figsize=(10,10))
plt.plot(grid_old_t, label='old_coord_volume_generator')
plt.plot(grid_t, 'r', label='new_coord_volume_generator')
plt.xticks(ticks = range(len(xticks)), labels=xticks)
plt.xlabel('batch_size')
plt.ylabel('time, sec.')